In [1]:
import torch
import torchvision
import pandas as pd
import numpy as np

In [2]:
import json
import os
import unicodedata

In [3]:
import random
from transformers import set_seed

In [4]:
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
set_seed(seed)

In [5]:
from tqdm import tqdm
import pymupdf
import pymupdf4llm
from collections import Counter

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig

from accelerate import Accelerator

import langchain

from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownTextSplitter, MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_teddynote.retrievers import KiwiBM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader

import peft
from peft import PeftModel

c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\exllama.py:12: UserWarning: AutoAWQ could not load ExLlama kernels extension. Details: DLL load failed while importing exl_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlama kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\exllamav2.py:13: UserWarning: AutoAWQ could not load ExLlamaV2 kernels extension. Details: DLL load failed while importing exlv2_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load ExLlamaV2 kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\gemm.py:14: UserWarning: AutoAWQ could not load GEMM kernels extension. Details: DLL load failed while importing awq_ext: 지정된 프로시저를 찾을 수 없습니다.
  warnings.warn(f"AutoAWQ could not load GEMM kernels extension. Details: {ex}")
c:\users\dohyeong\anaconda3\lib\site-packages\awq\modules\linear\gemv.py:11: UserWarning: AutoAWQ could not

In [55]:
class Opt:
    def __init__(self):
        self.model_configs = {
            'skt/kogpt2-base-v2': {
                'quantization_config': BitsAndBytesConfig(
                    load_in_8bit=True,
                    bnb_4bit_compute_dtype=torch.bfloat16,
                ),
                'torch_dtype': torch.float16,
                'max_token': 512,
            },
        }

        self.llm_model = 'skt/kogpt2-base-v2'
        self.llm_model_config = self.model_configs[self.llm_model]
        self.llm_peft = False
        self.llm_peft_checkpoint = None

        self.embed_models = ['distilbert-base-uncased', 'google/mobilebert-uncased']
        self.embed_model = self.embed_models[1]

        self.pdf_loader = 'pymupdf'

        self.base_directory = './'
        self.train_csv_path = os.path.join(self.base_directory, 'train.csv')
        self.test_csv_path = os.path.join(self.base_directory, 'test.csv')
        self.chunk_size = 256
        self.chunk_overlap = 16

        self.ensemble = True
        self.bm25_w = 0.5
        self.faiss_w = 0.5

        self.is_submit = True
        self.eval_sum_mode = False

        self.output_dir = 'test_results'
        self.output_csv_file = f"{self.llm_model.replace('/', '_')}_{self.embed_model.split('/')[1]}_pdf{self.pdf_loader}_chks{self.chunk_size}_chkovp{self.chunk_overlap}_bm25{self.bm25_w}_faiss{self.faiss_w}_mix_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)

    def to_json(self):
        return json.dumps(self.__dict__)

args = Opt()

In [56]:
def setup_llm_pipeline():
    tokenizer = AutoTokenizer.from_pretrained(args.llm_model)
    tokenizer.use_default_system_prompt = False

    model = AutoModelForCausalLM.from_pretrained(
        args.llm_model,
        quantization_config=args.llm_model_config['quantization_config'],
        torch_dtype=args.llm_model_config['torch_dtype'],
        device_map='auto',
        trust_remote_code=True
    )

    if args.llm_peft:
        model = PeftModel.from_pretrained(model, args.llm_peft_checkpoint)

    text_generation_pipeline = pipeline(
        model=model,
        tokenizer=tokenizer,
        task='text-generation',
        return_full_text=False,
        max_new_tokens=args.llm_model_config['max_token']
    )

    return HuggingFacePipeline(pipeline=text_generation_pipeline)

In [57]:
def normalize_path(path):
    return unicodedata.normalize('NFC', path)

def format_docs(docs):
    return '\n'.join([doc.page_content for doc in docs])

def process_pdf(file_path):
    md_text = pymupdf4llm.to_markdown(file_path)
    header_split = [
        ('#', 'header I'),
        ('##', 'header II'),
        ('###', 'header III'),
    ]

    md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=header_split, strip_headers=False)
    md_chunks = md_header_splitter.split_text(md_text)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=args.chunk_size, chunk_overlap=args.chunk_overlap
    )

    splits = text_splitter.split_documents(md_chunks)
    return splits

In [58]:
def create_vector_db(chunks, model_path, method='faiss'):
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

In [59]:
def process_single_pdf(pdf_path):

    chunks = process_pdf(pdf_path)
    
    db = create_vector_db(chunks, model_path=args.embed_model)
    
    kiwi_bm25_retriever = KiwiBM25Retriever.from_documents(chunks)
    faiss_retriever = db.as_retriever()

    retriever = EnsembleRetriever(
        retrievers=[kiwi_bm25_retriever, faiss_retriever],
        weights=[args.bm25_w, args.faiss_w],
        search_type='mmr'
    )

    del chunks, db, kiwi_bm25_retriever, faiss_retriever
    torch.cuda.empty_cache()

    return retriever

In [60]:
def process_questions_for_pdf(pdf_path, questions_df):

    retriever = process_single_pdf(pdf_path)

    llm_pipeline = setup_llm_pipeline()

    answers = []
    for _, row in questions_df.iterrows():
        question = row['Question']
        print(f"Generating answer for: {question}")

        result = llm_pipeline(question)
        # answers.append({
        #     'ID': row['SAMPLE_ID'],
        #     'Answer': result[0]['generated_text']
        # })

        if isinstance(result, list):
            answer_text = result[0]['generated_text'] if 'generated_text' in result[0] else result[0]
        elif isinstance(result, dict):
            answer_text = result.get('generated_text', result)
        else:
            answer_text = result 

        answers.append({
            'SAMPLE_ID': row['SAMPLE_ID'], 
            'Answer': answer_text
        })
    
    # 메모리 해제
    del retriever, llm_pipeline
    torch.cuda.empty_cache()

    return answers

In [61]:
def process_test_questions(df):

    all_answers = []
    unique_paths = df['Source_path'].unique()

    for path in tqdm(unique_paths, desc='Processing PDFs'):
        pdf_questions_df = df[df['Source_path'] == path]

        answers = process_questions_for_pdf(path, pdf_questions_df)
        all_answers.extend(answers)

    return all_answers

In [62]:
test = pd.read_csv('./test.csv')

In [63]:
test

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [64]:
train = pd.read_csv("./train.csv")
train

,SAMPLE_ID,Source,Source_path,Question,Answer
0,TRAIN_000,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부 재정체계는 어떻게 구성되어 있나요?,"2024년 중앙정부 재정체계는 예산(일반·특별회계)과 기금으로 구분되며, 2024년..."
1,TRAIN_001,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,2024년 중앙정부의 예산 지출은 어떻게 구성되어 있나요?,"2024년 중앙정부의 예산 지출은 일반회계 356.5조원, 21개 특별회계 81.7..."
2,TRAIN_002,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,기금이 예산과 다른 점은?,"기금은 예산과 구분되는 재정수단으로서 재정운영의 신축성을 기할 필요가 있을 때, 정..."
3,TRAIN_003,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"일반회계, 특별회계, 기금 간의 차이점은 무엇인가요?","일반회계는 특정 사업 운영 및 특정 세입으로 특정 세출을 충당하는데 사용되고, 특별..."
4,TRAIN_004,1-1 2024 주요 재정통계 1권,./train_source/1-1 2024 주요 재정통계 1권.pdf,"2024년 총수입은 얼마이며, 예산수입과 기금수입은 각각 몇 조원인가요?","2024년 총수입은 612.2조원이며, 예산수입은 395.5조원, 기금수입은 216..."
...,...,...,...,...,...
491,TRAIN_491,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,자치단체 보조금과 민간보조금은 각각 어떤 비율로 증가했는가?,"자치단체 보조금은 2019년 대비 2022년에 35% 증가하였고, 민간보조금은 10..."
492,TRAIN_492,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,한국의 재정금융안정계획이 주로 어떤 목표에 초점을 맞추어 있었는가?,한국의 재정금융안정계획은 통화량 조절과 물가안정이라는 단기적 목표에 초점을 맞추어 ...
493,TRAIN_493,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,1952년에 체결된 '한미경제조정협정'은 어떤 문제를 해결하기 위해 체결되었는가?,"원조물자 판매대금의 효과적 활용, 참전유엔군 경비지출을 위해 우리 정부에서 대여해 ..."
494,TRAIN_494,월간 나라재정 2023년 12월호,./train_source/월간 나라재정 2023년 12월호.pdf,"2023년 12월 IMF는 성장 전망을 어떻게 제시하고 있으며, 성장세의 둔화가 어...","IMF는 최근 세계 경제전망을 통해 작년 3.5%에서 올해 3%, 내년 2.9%로 ..."


In [65]:
if __name__ == "__main__":
    test_df = pd.read_csv(args.test_csv_path)
    answers = process_test_questions(test_df)
    print(answers)

    # 결과 저장
    output_path = os.path.join(args.output_dir, args.output_csv_file)
    pd.DataFrame(answers).to_csv(output_path, index=False)

Processing PDFs:   0%|                                                                           | 0/9 [00:00<?, ?it/s]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating answer for: 2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
Generating answer for: 중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
Generating answer for: 중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
Generating answer for: 2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
Generating answer for: 혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
Generating answer for: 재창업자금이 재도약지원자금으로 이관된 연도는 언제인가요?
Generating answer for: 창업기반지원과 신청 대상이 중복인 자금이 어떤 것이며, 이 자금이 폐지된 연도는 언제인가요?
Generating answer for: 혁신창업사업화자금(융자) 사업을 시행하는 주체는 누구인가요?
Generating answer for: 혁신창업사업화자금(융자) 사업 집행절차는 어떻게 되나요?


Processing PDFs:  11%|███████▎                                                          | 1/9 [02:57<23:36, 177.01s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating answer for: 부모급여 지원 사업의 목적은 무엇인가요?
Generating answer for: 부모급여(영아수당)의 2024년 확정된 예산은 몇백만원인가요?
Generating answer for: 부모급여 지원 사업은 어떤 법령상 근거를 갖고 추진되고 있나요?
Generating answer for: 영아수당 도입에 대한 추진경위는 어떻게 되나요?
Generating answer for: 부모급여 지원사업은 언제부터 시행되었나요?
Generating answer for: 보건복지부의 부모급여(영아수당) 지원 사업시행방법은 무엇이며, 사업 수혜자는 누구인가?


Processing PDFs:  22%|██████████████▋                                                   | 2/9 [04:56<16:42, 143.15s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Generating answer for: 재정성과관리제도는 어떤 측면에서 국정운영과 연결되는가?
Generating answer for: 성과관리의 실효성 강화를 위해 정부가 취한 조치는 무엇인가?
Generating answer for: 재정성과관리 관련 주요 쟁점은 무엇인가?
Generating answer for: 재정성과관리가 왜 중요한가?
Generating answer for: 재정성과관리는 무엇을 목표로 하는가?
Generating answer for: 어떤 국제기구들이 사업을 기준으로 예산을 나누어 성과 정보를 생산하고 있는가?
Generating answer for: 재정성과관리의 목적은 무엇인가?
Generating answer for: 2021년 「국가재정법」 개정으로 어떤 규정이 신설되었는가?
Generating answer for: 성과관리제도는 지출 구조조정을 위해 어떤 방향으로 추진되고 있는가?
Generating answer for: 재정사업 자율평가의 목적은 무엇이며, 어떤 방식으로 제도 개선이 이루어졌는가?
Generating answer for: 2016년 재정성과관리제도의 환류 개선사항은 무엇인가?
Generating answer for: 2018년도에 재정성과관리제도 개선사항과, 이로 인해 어떤 효과가 발생했는가?
Generating answer for: 재정사업 자율평가의 전면 개편을 통해 어떤 중점 추진과제가 제시되었는가?
Generating answer for: 재정성과관리제도의 중요성과 국정운영과의 연결성은 무엇인가?
Generating answer for: 재정성과관리체계 강화를 위해 정부가 어떤 제도를 제시했으며, 재정성과관리는 무엇을 목표로 하는가?
Generating answer for: 재정성과관리는 어떤 과정에서 수행되며, 무엇을 증진하기 위해 활동하는가?
Generating answer for: 재정성과관리의 목적과 우리나라의 재정성과관리제도가 프로그램 예산제도로 전환된 시기는?
Generating an

Processing PDFs:  89%|██████████████████████████████████████████████████████████▋       | 8/9 [25:20<03:50, 230.83s/it]No sentence-transformers model found with name google/mobilebert-uncased. Creating a new one with mean pooling.
c:\users\dohyeong\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Generating answer for: 우리나라에서는 언제부터 발생주의 기준을 적용한 국가결산보고서에서 우발부채를 보고하고 있는가?
Generating answer for: 우발부채 관련 주요 쟁점은 무엇인가?
Generating answer for: 우발부채의 관리는 왜 중요한 이슈로 여겨지는가?
Generating answer for: 우발부채와 부채의 차이점은 무엇인가?
Generating answer for: 발생주의와 현금주의의 차이는 무엇인가?
Generating answer for: 채무지속가능성분석은 어떤 목적을 가지고 도입되었는가?
Generating answer for: 의제의무란 무엇인가?
Generating answer for: 국제통화기금이 재정통계 작성의 국제기준을 제시하기 위해 발간한 매뉴얼은 무엇인가?
Generating answer for: 계류중인 소송사건이란 무엇인가?
Generating answer for: 최소운영수입보장(BTO 등) 제도란 무엇을 의미하는가?
Generating answer for: 우발부채에 대한 내용으로 대표적으로 어떤 사항이 해당되는가?
Generating answer for: GFSM2014에서는 우발부채를 어떻게 구분하는가?
Generating answer for: GFSM은 몇 차례의 개정을 거쳤으며, 어떠한 목적으로 GFSM 2001이 개정되었는가?
Generating answer for: 표준화 보증이란 무엇인가?
Generating answer for: 표준화 보증에서 공공부문의 우발부채는 어떻게 인식되는가?
Generating answer for: 재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
Generating answer for: 미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
Generating answer for: 국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게 다른가요?
Generating answer for: 우발부채

Processing PDFs: 100%|██████████████████████████████████████████████████████████████████| 9/9 [31:46<00:00, 211.86s/it]

[{'SAMPLE_ID': 'TEST_000', 'Answer': '?\n(한국판 뉴딜정책) "한국판 뉴딜프로젝트"라고 말할 수 있다.\n창업도전지원시스템으로 창업자의 부담을 줄이고, 신기술이 사업화될 길을 열어주는 것이 아니라, 새로운 사업아이템을 만들어내면 된다.\n그리고 해당 기업에 대한 시뮬레이션과 현장실습을 바탕으로 후속사업을 추진하기 위해서는 기업 스스로 더 성장가능성이 있고 지속적인 성장을 이룰 수 있는 새로운 창업 아이템을 창출해야 한다.\n이제는 창업 후 2년, 4년, 5년, 5년의 생존기간이 짧아지고 성공가능성을 높일 수도 있다는 것을 인식해야만 한다.\n그런데 창업시 고려해야 할 대목은 결국 창업 실패 위험이다.\n창업에 실패한 후 창업자의 어려움을 어떻게 해소할 것인가 하는 것이다.\n창업 후 2~3년이 지나야 사업성이 있고, 사업자가 실패할 경우, 창업자는 자신의 사업 아이템을 그대로 유지하고 새로운 것을 시도하는 것이 좋다.\n창업 후 3년이 지나야 사업 성사가 있는 것도 아니고, 창업 후 3년이 지나야 성장성이 있기 때문이다.\n창업의 목표는 실패에 대한 책임감이다.\n하지만 창업 후 1년이 지나고 3년이 지나야 성과창출 여부를 판단할 수 있다는 것이 창업 전부의 얘기다.\n실사보다 실사 보다 실사보단 최종 판단이 더욱 중요하다.\n창업 후 4~5년이 지나야 성공 확률이 있으며, 향후 1년 안에 창업이 성공할 가능성은 있다.\n또한 창업을 계획하고 있는 과정에서 필요한 자금을 충분히 준비하지 못했다면 창업을 포기해야 한다.\n창업 후 10년을 지나야 사업성공률이 있는 것도 아니다.\n창업을 준비하는 과정에서 초기 투자를 하면 실패에 대한 책임을 완전히 회피하기도 하고 막연한 불안감을 가질 수도 있는 것이다.\n따라서 초기 투자자금이 충분히 준비되지 못한다면 창업 후 3년 후에야 창업할 수 있는 사업아이템과 성공 가능성이 있다.\n하지만 창업 후에는 사업아이템의 성공 확률이 현재 시장상황과 비교하면 오히려 낮다.\n다만, 창업 후 3년

In [69]:
test

,SAMPLE_ID,Source,Source_path,Question
0,TEST_000,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2022년 혁신창업사업화자금(융자)의 예산은 얼마인가요?
1,TEST_001,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업목적은 무엇인가요?
2,TEST_002,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,중소벤처기업부의 혁신창업사업화자금(융자) 사업근거는 어떤 법률에 근거하고 있나요?
3,TEST_003,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,2010년에 신규 지원된 혁신창업사업화자금은 무엇인가요?
4,TEST_004,중소벤처기업부_혁신창업사업화자금(융자),./test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf,혁신창업사업화자금 중 2020년에 신규 지원된 자금은 무엇인가요?
...,...,...,...,...
93,TEST_093,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,재정정책에서 공적보증채무와 다른 일회성 보증은 어떻게 구분되는가?
94,TEST_094,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,미래사회보장급여에 대한 순의무란 무엇을 의미하는가?
95,TEST_095,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,국가결산보고서와 지방자치단체 회계기준에서 우발부채에 대한 용어 및 회계처리가 어떻게...
96,TEST_096,「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》,./test_source/「FIS 이슈 & 포커스」(신규) 통권 제1호 《우발부채》...,"우발부채란 무엇이며, 그 관리가 왜 중요한가?"


In [68]:
pd.read_csv("./test_results/skt_kogpt2-base-v2_mobilebert-uncased_pdfpymupdf_chks256_chkovp16_bm250.5_faiss0.5_mix_submission.csv")

,SAMPLE_ID,Answer
0,TEST_000,"?\n(한국판 뉴딜정책) ""한국판 뉴딜프로젝트""라고 말할 수 있다.\n창업도전지원시..."
1,TEST_001,"""라고 물었다.\n이에 대해 ""중기청에서 중소기업에 대한 금융지원 프로그램을 운영하..."
2,TEST_002,"""\n이 사업용도로는 사업자가 자금 조달이 어려워 투자나 투자를 꺼리는 것이 아니라..."
3,TEST_003,"?\n우리 사회 전반으로 확산되길 바랍니다. 지난 10월 초, 한 대학에서는 수학과..."
4,TEST_004,“ 이렇게 간단한 질문을 던지기가 쉽지 않을 것입니다.\n무슨 뜻인지 알아야 하지 ...
...,...,...
93,TEST_093,""" 라는 연구용 논문을 통해 이와 같은 가설을 뒷받침한다. 최근 스마트폰과 태블릿P..."
94,TEST_094,?\n어~ 사회보험을 도입되면 국민들이 받는 보험료가 어느 정도 증가하게 될것입니다...
95,TEST_095,' 등과 같이 문제되는 사항에 대해서는 그 원인을 명확히 해명하지 않고 있다.\n이...
96,TEST_096,"""라며 ""아무리 잘 관리하면 오히려 부실의 우려가 커지고 그로 인한 금융시스템의 위..."


In [67]:
pd.read_csv("./test_results/gpt2_mobilebert-uncased_pdfpymupdf_chks128_chkovp8_bm250.5_faiss0.5_mix_submission.csv").loc[0, 'Answer']

' (Gukasanan, "음엽엉가 최 보의쓜"음꧀ 맅은 민디지, "엽가엉을 보의쓜"음꧀ 민디�엽가 Tietnam, "엽� 자다/음껀(, "읏껌"다, "이�'